## RNA-seqカウントデータの前処理
本講義では、pandasを用いてRNA-seqカウントデータの処理を行います。

#### RNA-seqとは
RNA-seqとは, mRNAやmiRNAの配列をシーケンスして、発現量の定量や新規転写産物の同定を行う手法です。<br>
シーケンスで得られたデータ（リード）は, 以下のようなステップで解析します。<br>
1. リードのトリミング
2. ゲノム配列へのマッピング
3. マッピングされたリード数を数える
4. サンプル毎の総リード数の違いや、遺伝子配列長の違いを補正（正規化） 
5. 遺伝子毎の発現量を同定、比較

本講義では上記ステップのうち4以降を扱います。

####  本講義で用いるRNA-seqデータについて
本講義では, 疾患モデルマウスのRNA-seqデータセット（GSE190806）を使用します。<br>

Impaired KDM2B-mediated PRC1 recruitment to chromatin causes defective neural stem cell self-renewal and ASD/ID-like behaviors [RNA-Seq]<br>
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE190806<br>
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE190807<br>
（GSE190806とGSE190807のRNA-seqは同じSRAアクセッション）

このRNA-seqデータはINSDCのSRAデータベースにアクセッション番号 SRR17223720-SRR17223725 としてアーカイブされています。<br>
SRR17223720-SRR17223722 Neural progenitor cells(C57BL/6J由来), wild-type<br>
SRR17223723-SRR17223725 Neural progenitor cells(C57BL/6J由来), Kdm2b one allele mutant<br>
このほか, サンプルの詳細は以下の論文を参照してください。<br>

Impaired KDM2B-mediated PRC1 recruitment to chromatin causes defective neural stem cell self- renewal and ASD/ID-like behaviors<br>
DOI:https://doi.org/10.1016/j.isci.2022.103742


アーカイブされたデータをダウンロードして遺伝子ごとのカウントデータにするまでの手順と, 使用したソフトウェアは以下の通りです。<br>

1. シークエンスデータの取得 fasterq-dump v3.0.0 ( https://github.com/ncbi/sra-tools )
2. リファレンス:GRCm39のcDNA配列（ Mus_musculus.GRCm39.cdna.all.fa, http://asia.ensembl.org/Mus_musculus/Info/Index )
3. リードの前処理 (リードトリミング、クオリティフィルタリングなど) fastp v0.23.2 ( https://github.com/OpenGene/fastp )
4. 転写産物の定量 kallisto v0.48.0 ( https://github.com/pachterlab/kallisto )

これらの処理の詳しい内容については本講習会では扱いません。何卒ご了承ください。

#### 本講義で行うRNA-seqデータの処理

- データの読み込み
- カウントデータの正規化（ RPM/FPM, FPKM, TPM)
- サンプル間のクラスタリング　
- 遺伝子のアノテーション( transcript_idとgene_idの対応、gene_idに対応したdescriptionの付与 )
- 発現量比 ( fold-change )の計算

### 1.データファイルの読み込み

#### 準備
pandas を pd としてインポートします。同様に numpy を np としてインポートします。

In [ ]:
# pandasのimport
import pandas as pd
import numpy as np

SRR17223720-SRR17223725 のカウントデータをまとめたファイル ( counts_kalisto.tsv ) のパスを指定します。<br>
同様に、アノテーションデータとしてtranscript_idとgene_idを紐づけたファイル( transcript_name.tsv )、<br>
gene id と description を紐づけたファイル ( description.tsv ) のパスも指定します。<br>
アノテーションデータはGRCm39のGFFファイル( Mus_musculus.GRCm39.108.gff3 ) から必要な情報を抽出して作成しました。

In [ ]:
count_file = 'data/counts_kallisto.tsv'
transcript_name_file = 'data/transcript_name.tsv'
description_file = 'data/description.tsv'

#### カウントデータファイルについて<br>
counts_kallisto.tsvを開くと以下のようになっています。<br>
```
target_id	eff_length	SRR17223720	SRR17223721	SRR17223722	SRR17223723	SRR17223724	SRR17223725
ENSMUST00000178537.2	6.74193	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000178862.2	7.65825	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000196221.2	5.34639	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000179664.2	6.27959	0.0	0.0	0.0	0.0	0.0	0.0
ENSMUST00000177564.2	8.56364	0.0	0.0	0.0	0.0	0.0	0.0
... 以下省略
```
1行目は列タイトルを表すヘッダー行です。<br>
2行目以降からがデータ行です。一番左の列が遺伝子idになっているのでこれをインデックスに用います。

`pd.read_table()` メソッドの`index_col`オプションを指定して読み込みます。<br>

- index_col...インデックスとして用いる列を数字で指定します。

In [ ]:
df = pd.read_table(count_file, index_col=0)

`pd.read_table(path/to/file)` のかわりに`pd.read_csv(path/to/file,sep='\t') `を用いることもできます。<br>
`sep='\t'`は`delimiter='\t'` も使えます。

#### データの概観
データを概観してみましょう。

In [ ]:
# head()で列名とインデックスが正しく読み込まれているかを確認します
df.head()

In [ ]:
# 'eff_length'列だけ確認
df.eff_length.head()

カウントデータファイルの行数と列数を確認します。

In [ ]:
# データ件数を確認
df.shape

#### 列名を変更する

In [ ]:
# 列名を変更するための対応表
names = {'SRR17223720': 'wt_1',
         'SRR17223721': 'wt_2',
         'SRR17223722': 'wt_3',
         'SRR17223723': 'mutant_1',
         'SRR17223724': 'mutant_2',
         'SRR17223725': 'mutant_3'}

`rename()` を `axis=1` を適用して使い、列名を変更します。

In [ ]:
df = df.rename(mapper=names, axis=1)

`rename()` を使わなくても、既存の列を別名でコピーした後で元の列を削除、という方法でも可能です。
```
df['wt_1'] = df['SRR17223720']
del df['SRR17223720']
```

In [ ]:
df.head()

カウントデータ部分のみを切り出し

In [ ]:
df_count = df[['wt_1','wt_2','wt_3','mutant_1','mutant_2','mutant_3']]

In [ ]:
# スライスで指定する場合　
df.iloc[:, 1:7]

#### ファイルの保存
保存用に`output/`フォルダを用意し, estimate_countのみのデータを`count_raw.tsv`として保存します。

In [ ]:
# estimate_countのみのデータの保存
df_count.to_csv('output/count_raw.tsv', sep = '\t')

シェルコマンド head を使って確認してみます。<br>
`%%bash`でJupyter notebookのセル内でbashコマンドを実行させることができます。

In [ ]:
%%bash
head output/count_raw.tsv

### 2. カウントデータの正規化
#### Step1. リード数で正規化 (RPM/FPM)

100万リードあたりのカウント数に揃えます。<br>
RPM = reads per million, FPM = fragments per million<br>
ほぼ同じ意味で用いられているので, 本講義では「FPM」を使います。

カウントデータをいったん別のデータフレームとしてコピーしておきます。

In [ ]:
df_tmp = df_count.copy()

In [ ]:
df_tmp.iloc[6085:6090,] # head()だと0.0ばかりになる

In [ ]:
# リード数の合計　sum()を使って計算
sum_count = df_tmp.sum()

In [ ]:
sum_count

In [ ]:
# 100万リードあたりに揃える
df_tmp = 10**6 * df_tmp / sum_count

In [ ]:
df_tmp.iloc[6085:6090,]

リード数の合計が100万に揃っていることを確認します。

In [ ]:
df_tmp.sum()

`normalize_per_million_reads()` として関数化しておきます。

In [ ]:
def normalize_per_million_reads(df):
    sum_count = df.sum()
    return 10**6 * df / sum_count

カウントデータに適用

In [ ]:
df_count_fpm = normalize_per_million_reads(df_count)

確認

In [ ]:
df_count_fpm.sum()

FPM正規化を行った結果を `count_fpm.tsv` として保存します。

In [ ]:
df_count_fpm.to_csv('output/count_fpm.tsv', sep='\t')

#### Step2. 遺伝子長による正規化 (RPKM/FPKM)
上で求めたFPMをさらに遺伝子長で割ってFPKMを求めます。<br>
FPKM = fragments per kilobase of exon per million reads mapped <br>
今回用いたsingle-endの場合、RPKM = reads per kilobase of exon per million reads mapped と呼ばれますが, <br>
FPKM/RPKMはほぼ同じ意味で用いられています。

各遺伝子の長さを抽出しておきます。<br>
kallistoでは実際の配列長（abundance.tsvの`length`列）ではなく、`eff_length`列でTPMを計算しているので、<br>
本講義でも`eff_length`列の値を用います。<br>
参考　次世代シーケンサーデータの解析手法 第 15 回 RNA-seq 解析（その3）<br>
DOI: https://doi.org/10.4109/jslab.31.25


In [ ]:
gene_length = df['eff_length']

In [ ]:
gene_length

下のように書くとうまくいきません。（縦方向にブロードキャストされてしまうため）

In [ ]:
# df_tmp / gene_length

一列ずつ計算することは可能です。

In [ ]:
# df_tmp["batch_1"] / gene_length * 10**3

##### for ループを使う方法
データフレームをforループで回すと、列名が取得できるのでそれを利用します。

In [ ]:
# テスト用にDataFrameをコピーしてから
df_tmp = df_count_fpm.copy()

for col_name in df_tmp:
    df_tmp[col_name] = df_tmp[col_name] / gene_length * 10**3

In [ ]:
df_tmp.iloc[6085:6090,]

`iteritems()` を利用する方法もあります。

In [ ]:
df_tmp = df_count_fpm.copy()

for col_name, col in df_tmp.iteritems():
    df_tmp[col_name] = col / gene_length * 10**3

In [ ]:
df_tmp.iloc[6085:6090,]

##### データフレームを転置してから計算する方法

In [ ]:
# テスト用にFPMをコピー
df_tmp = df_count_fpm.copy()

# df_tmpを転置してFPMを遺伝子長で割り, 1000をかける
df_tmp = df_tmp.T / gene_length * 10**3

# 戻す（もう一度転置する）
df_tmp = df_tmp.T

In [ ]:
df_tmp.iloc[6085:6090,]

##### applyを使い各列に関数を適用する方法
`divide_by_length()`という関数を作り、`apply()` で適用します。

In [ ]:
# 列を入力とし、各要素を遺伝子長で割る処理を行う関数を定義
def divide_by_length(S):
    return S / gene_length * 10**3

In [ ]:
# テスト用にFPMをコピー
df_tmp = df_count_fpm.copy()

# applyでdivide_by_length()関数を適用して先頭だけ確認
df_tmp.apply(divide_by_length).iloc[6085:6090,]

`pandas.DataFrame`メソッドの `divide()` を使用する方法もあります。

In [ ]:
# テスト用にFPMをコピー
df_tmp = df_count_fpm.copy()

df_tmp = df_tmp.divide(gene_length, axis='index') * 10**3
df_tmp.iloc[6085:6090,]

データフレームを転置させて計算する方法を `normalize_per_kilobase()` として関数化しておきます。

In [ ]:
def normalize_per_kilobase(df, gene_length):
    df_tmp = df.copy()
    df_tmp = (df.T * 10**3 / gene_length).T 
    return df_tmp

In [ ]:
df_count_fpkm = normalize_per_kilobase(df_count_fpm, gene_length)

In [ ]:
df_count_fpkm.iloc[6085:6090,]

In [ ]:
# 保存する
df_count_fpkm.to_csv('output/count_fpkm.tsv', sep='\t')

#### Step3. TPM 正規化
TPM = transcripts per million <br> 
TPM の説明については以下のページが詳しいです。 https://bi.biopapyrus.jp/ <br> 

FPKM/RPKM のときとは逆に、長さ1,000bpあたりのリード数を求めてから、総リード数を100万に揃えます。

In [ ]:
# テスト用にカウントデータをコピー
df_tmp = df_count.copy()

In [ ]:
df_tmp = normalize_per_kilobase(df_tmp, gene_length) #長さ1,000bpあたりのリード数
df_tmp = normalize_per_million_reads(df_tmp) #総リード数を100万に揃える

In [ ]:
df_tmp.iloc[6085:6090,]

RPKM/FPKMと違い、合計が100万となっています。

In [ ]:
df_tmp.sum()

`normalize_tpm()` として関数化しておきます。

In [ ]:
def normalize_tpm(df, gene_length):
    df_tmp = df.copy()
    df_tmp = normalize_per_kilobase(df_tmp, gene_length)
    df_tmp = normalize_per_million_reads(df_tmp)
    return df_tmp

In [ ]:
df_count_tpm = normalize_tpm(df_count, gene_length)

In [ ]:
df_count_tpm.sum() #  確認

In [ ]:
# 保存
df_count_tpm.to_csv('output/count_tpm.tsv', sep='\t')

* 参考）numpyを使った高速バージョンとの比較 <br>

`%%timeit`はコードの時間計測を何回か試し, その中で最速の時間と平均値を返すマジックコマンドです。

In [ ]:
%%timeit
# pandasで実装したもの
normalize_tpm(df_count, gene_length)

In [ ]:
# valuesによりnumpy.ndarrayとして数値データを抽出
counts = df_count.values
length =gene_length.values

In [ ]:
%%timeit
# numpyで計算
# 長さで正規化。行方向へbroadcastを行うため、reshapeしておく必要がある
counts_tmp = counts / length.reshape(-1, 1) * 1000
# カウント数の各列の合計を求めておく
sum_count = counts_tmp.sum(axis=0)
# 100万カウントに揃える
tpm = counts_tmp / sum_count * 1000000

### 3. TPM正規化したデータのクラスタリング
matplotlib と scipy の必要モジュールをインポートします。


In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [ ]:
# 転置
tpm_t = df_count_tpm.T
tpm_t.iloc[:,0:3]

In [ ]:
# クラスタリング
linkage_result = linkage(tpm_t, method='average', metric='correlation')

# 結果の可視化
plt.figure(num=None, figsize = (12,4), facecolor='w', edgecolor='k')
dendrogram(linkage_result,labels = list(tpm_t.index) )
plt.show()

参考：PCA

In [ ]:
import sklearn
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(tpm_t)

# データを主成分空間に写像
pca_row = pca.transform(tpm_t)

# 寄与率を求める
pca_col = ["PC{}".format(x + 1) for x in range(len(tpm_t.index))]
df_con_ratio = pd.DataFrame([pca.explained_variance_ratio_], columns = pca_col)
print(df_con_ratio.head())

# PC1とPC2でplot
plt.figure(figsize=(3, 3))
plt.scatter(pca_row[:, 0], pca_row[:, 1], alpha=0.8, c=(1,1,1,2,2,2))
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
annotations = tpm_t.index
for i, label in enumerate(annotations):
    plt.annotate(label, (pca_row[i, 0], pca_row[i, 1]))
plt.show()

### 4. 遺伝子アノテーションをつける

#### Step1. アノテーションファイルを読み込む

transcript_name.tsvを開くと以下のようになっています。<br>

```
ENSMUST00000070533.5	Xkr4-201	ENSMUSG00000051951
ENSMUST00000208660.2	Rp1-202	ENSMUSG00000025900
ENSMUST00000027032.6	Rp1-201	ENSMUSG00000025900
ENSMUST00000027035.10	Sox17-201	ENSMUSG00000025902
ENSMUST00000195555.2	Sox17-208	ENSMUSG00000025902
...
```

DataFrame名を `transcript_name` として、 <br>
ヘッダー（列名）は `names=['transcript_id', 'transcript_name', 'gene_id']`として読み込みます。

In [ ]:
transcript_name = pd.read_table(transcript_name_file, names=['transcript_id', 'transcript_name', 'gene_id'])
print(transcript_name.shape)
transcript_name.head()

description.tsvを開くと以下のようになっています。<br>

```
ENSMUSG00000102693	4933401J01Rik	RIKEN cDNA 4933401J01 gene [Source:MGI Symbol%3BAcc:MGI:1918292]
ENSMUSG00000051951	Xkr4	X-linked Kx blood group related 4 [Source:MGI Symbol%3BAcc:MGI:3528744]
ENSMUSG00000103377	Gm37180	predicted gene%2C 37180 [Source:MGI Symbol%3BAcc:MGI:5610408]
ENSMUSG00000104017	Gm37363	predicted gene%2C 37363 [Source:MGI Symbol%3BAcc:MGI:5610591]
ENSMUSG00000103025	Gm37686	predicted gene%2C 37686 [Source:MGI Symbol%3BAcc:MGI:5610914]
...
```

DataFrame名を `gene_desctiption` として、<br>
ヘッダー（列名）は `names=['gene_id', 'gene_name','description']`として読み込みます。

In [ ]:
gene_description = pd.read_table(description_file, names=['gene_id', 'gene_name','description'])

In [ ]:
print(gene_description.shape)
gene_description.head()

#### Step2. 2つのアノテーションデータを連結
2つのDataFrame `transcript_name` と `gene_description` を連結し`gene_products`　という新しいDataFrameを作ります。

In [ ]:
gene_products = pd.merge(transcript_name, gene_description, on='gene_id', how='left') 


transcript_idのアノテーションが決まればよいので　`how='left'` としました。

In [ ]:
print(gene_products.shape)
gene_products.head()

#### Step3. アノテーションデータとカウントデータを連結
2つのDataFrame `df` と `gene_products` を連結します。<br>
`gene_products` には mRNA のデータしか含まれていないので、rRNA などのデータはこの時点で除かれます。<br>
ただし、`gene_name`列の値が `mt-` で始まるミトコンドリア遺伝子が含まれています。

In [ ]:
df_with_product = df.copy()
df_with_product['transcript_id'] = df_with_product.index
df_with_product.head()

In [ ]:
df_with_product = pd.merge(df_with_product, gene_products, on='transcript_id', how='inner')

In [ ]:
# concatを使った場合
# pd.concat([gene_products, df], axis=1, join="inner")

In [ ]:
# transcript_id列をindexにする
df_with_product.set_index('transcript_id', inplace=True)

アノテーション付きカウントデータを`count_preprocessed.tsv`として保存します。<br>

In [ ]:
# アノテーション付きデータの保存
df_with_product.to_csv('output/count_preprocessed.tsv', sep = '\t')

TPMデータにもアノテーション付与

In [ ]:
df_tpm_with_product = df_count_tpm.copy()
df_tpm_with_product['transcript_id'] = df_tpm_with_product.index
df_tpm_with_product = pd.merge(df_tpm_with_product, gene_products, on='transcript_id', how='inner')
df_tpm_with_product.set_index('transcript_id', inplace=True)

df_tpm_with_product.head()

In [ ]:
# アノテーション付きデータの保存
df_tpm_with_product.to_csv('output/tpm_with_product.tsv', sep = '\t')

### 5. 発現変動遺伝子を抽出する

TPM正規化データを用います。<br>
wild typeの平均を求めます。

In [ ]:
df_count_tpm['wt'] = (df_count_tpm['wt_1'] + df_count_tpm['wt_2'] + df_count_tpm['wt_3']) / 3

mutantの平均を求めます。

In [ ]:
df_count_tpm['mutant'] = (df_count_tpm['mutant_1'] + df_count_tpm['mutant_2'] + df_count_tpm['mutant_3']) / 3

発現変動をlog2 fold として求めます。<br>
0 での除算を防ぐため、分母に微小な値を加えています。<br>

In [ ]:
df_count_tpm['log2fold'] = df_count_tpm['mutant'] / (df_count_tpm['wt'] + 10**-6)
df_count_tpm['log2fold'] = df_count_tpm['log2fold'].apply(np.log2)

In [ ]:
df_count_tpm.iloc[6085:6090,]

必要部分のみ抜き出し、productと結合します。
`df_count_tpm` から'wt', 'mutant', 'log2fold'の列を抜き出し`diff_ex`とします。

In [ ]:
diff_ex = df_count_tpm.copy()
diff_ex = diff_ex[['wt', 'mutant', 'log2fold']]
diff_ex ['transcript_id']= diff_ex.index

In [ ]:
diff_ex = pd.merge(diff_ex, gene_products, on = 'transcript_id', how = 'right')
diff_ex.set_index('transcript_id', inplace = True)

In [ ]:
diff_ex.head()

カウント数が0であるデータを除きます。

In [ ]:
diff_ex = diff_ex[diff_ex['wt'] > 0]
diff_ex = diff_ex[diff_ex['mutant'] > 0]

In [ ]:
diff_ex.shape

In [ ]:
diff_ex.head()

`diff_ex.sort_values()`を使い`log2fold`の降順に並びかえます。

In [ ]:
diff_ex = diff_ex.sort_values(by = 'log2fold',ascending = False)

発現変動遺伝子の上位を表示してみましょう。

In [ ]:
# mutant > wt の上位5番目まで表示
diff_ex.head()

In [ ]:
# wt > mutant の上位5番目まで表示
diff_ex.tail()